Seleccionamos la ruta principal de trabajo

In [1]:
import os
from pathlib import Path

root_directory = Path("C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main")
os.chdir(root_directory)
print(root_directory)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main


## Imports


In [2]:
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint


from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim, Patchcore

In [3]:
datamodule = MVTec(num_workers=0)
datamodule = MVTec(category="bottle")
datamodule.prepare_data()  
datamodule.setup()  
i, data = next(enumerate(datamodule.val_dataloader()))
print(data.keys())

dict_keys(['image_path', 'label', 'image', 'mask'])


Let's check the shapes of the input images and masks.


In [4]:
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

In [5]:
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",  
    devices=1,
    logger=False,
)

In [6]:
model = Patchcore()
datamodule2 = MVTec(num_workers=5)

In [7]:
engine = Engine(task=TaskType.SEGMENTATION)
engine.fit(model=model, datamodule=datamodule2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax                   | 0     
3 | image_threshold       | F1AdaptiveThreshold      | 0     
4 | pixel_threshold       | F

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.


## Validation


In [8]:
test_results = engine.test(
    model=model,
    datamodule=datamodule2,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Patchcore\MVTec\bottle\v3\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Patchcore\MVTec\bottle\v3\weights\lightning\model.ckpt
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9922077655792236     │
│       image_F1Score       │    0.9927536249160767     │
│        pixel_AUROC        │    0.5409324765205383     │
│       pixel_F1Score       │    0.2682372033596039     │
└───────────────────────────┴───────────────────────────┘

## OpenVINO Inference

Now that we trained and tested a model, we could check a single inference result using OpenVINO inferencer object. This will demonstrate how a trained model could be used for inference.


Before we can use OpenVINO inference, let's export the model to OpenVINO format first.


In [9]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\anomalib\data\transforms\center_crop.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if image.numel() == 0:
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\anomalib\data\transforms\center_crop.py:61: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if crop_height > image_height or crop_width > image_width:
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\anomalib\data\transforms\center_crop.py:41: TracerWarning: torch.tensor results are registered as constants in the trace. You can safe

WindowsPath('C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/results/Patchcore/MVTec/bottle/latest/weights/openvino/model.xml')

### Load the OpenVINO Model

By default, the output files are saved into `results` directory. Let's check where the OpenVINO model is stored.


In [11]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Patchcore\MVTec\bottle\latest


In [12]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [13]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path, 
    metadata=metadata,  
    device="CPU",  
)

In [14]:
print(openvino_model_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Patchcore\MVTec\bottle\latest\weights\openvino\model.bin


In [15]:
import os

# Define la ruta al  que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/broken_large/"

# Vamos los archivos del directorio
files = os.listdir(directory)

# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")

Archivo: 000.png, Score: 0.6717997455621605, Label: 1
Archivo: 001.png, Score: 0.7422591754767242, Label: 1
Archivo: 002.png, Score: 0.6957970356984249, Label: 1
Archivo: 003.png, Score: 0.6045903684559177, Label: 1
Archivo: 004.png, Score: 0.7171490707469648, Label: 1
Archivo: 005.png, Score: 0.6577599164447672, Label: 1
Archivo: 006.png, Score: 0.6115302488869789, Label: 1
Archivo: 007.png, Score: 0.5469422124200877, Label: 1
Archivo: 008.png, Score: 0.8283884485947537, Label: 1
Archivo: 009.png, Score: 0.7617970816115234, Label: 1
Archivo: 010.png, Score: 0.6168232033568751, Label: 1
Archivo: 011.png, Score: 0.6856575077742602, Label: 1
Archivo: 012.png, Score: 0.5907019534702127, Label: 1
Archivo: 013.png, Score: 0.7801916525032465, Label: 1
Archivo: 014.png, Score: 0.7968736317937967, Label: 1
Archivo: 015.png, Score: 0.7012833536776805, Label: 1
Archivo: 016.png, Score: 0.6310347276436654, Label: 1
Archivo: 017.png, Score: 0.6254857168149456, Label: 1
Archivo: 018.png, Score: 0.5

In [16]:
import os

# Define la ruta al directorio que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/good/"

# Vamos los archivos del directorio
files = os.listdir(directory)
# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")

Archivo: 000.png, Score: 0.47309330582070735, Label: 0
Archivo: 001.png, Score: 0.5012218499444336, Label: 1
Archivo: 002.png, Score: 0.4763194111496491, Label: 0
Archivo: 003.png, Score: 0.5255426457538559, Label: 1
Archivo: 004.png, Score: 0.5058303359071644, Label: 1
Archivo: 005.png, Score: 0.6211423377066936, Label: 1
Archivo: 006.png, Score: 0.582072140708122, Label: 1
